In [1]:
import sys
import numpy as np
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Poisson Problem at L Domain 
# 3-noded Linear Triangular Elements, $P_1$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
ELEMENT = ("Triangle", 1)
BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'entire_boundary'}]
MATERIAL = [{'type': 'material_constant_a', 'tag': 201, 'value': 1, 'name': 'entire_domain'}]

# Create mesh from file geometry
create_domain.L_geometry(ELEMENT, BOUNDARY, lc=1, view_mesh=True)
mesh_data = read_mesh.get_data(ELEMENT, BOUNDARY, MATERIAL, model='L', info_mode=False)

Model L_domain_Triangle1 (2D)
Info     : 13 geometric entities
Info     : 2 Physical Groups
Info     : 17 nodes in total
Info     : 20 2-D elements in total


### Mesh Data Dictionaries

In [3]:
mesh_nodes = mesh_data['nodes_data']; mesh_nodes[:1]

[{'TagID': 1,
  'global_coord': (0.0, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}}]

In [4]:
conn_data = mesh_data['conn_data']; conn_data[:1]

[{'TagID': 13,
  'conn_list': [10, 13, 16],
  'material': {'type': 'material_constant_a',
   'tag': 201,
   'value': 1,
   'name': 'entire_domain'}}]

In [5]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

### Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$

In [6]:
graph_results.plot_mesh(mesh_data, ELEMENT, model='L')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p13_L_domain_poisson\pre_processing\pictures\L_meshed_domain_Triangle1.svg


<figure>
    <img src="pre_processing/pictures/L_meshed_domain_Triangle1.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

### Assembly Matrices
#### `assembly.global_nodes_coordinates()` 

In [7]:
ai, xi, yi = matrices_assembly.global_nodes_coordinates(0, mesh_data)
print(f"'a_1' Global coordinates: {ai}")
print(f"'x_1' x-coordinates: {xi}")
print(f"'y_1' y-coordinates: {yi}")

'a_1' Global coordinates: [(2.752797989558076e-12, 1.0), (-0.3916417253665321, 0.2820677427439965), (0.1178335817045444, 0.4041559357628871)]
'x_1' x-coordinates: [2.752797989558076e-12, -0.3916417253665321, 0.1178335817045444]
'y_1' y-coordinates: [1.0, 0.2820677427439965, 0.4041559357628871]


#### `map_to_physical_coordinates()`

In [8]:
xi_master = (1, 0)
xg, yg = matrices_assembly.isomapping_to_global_coordinates(ai, xi_master, ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg}, {yg})")

'e_1' Master coordinates: (1, 0) --> Global coordinates: (-0.3916417253665321, 0.2820677427439965)


#### `apply_physics()`

In [9]:
mesh_data = ps.apply_physics(mesh_data, ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 13,
  'conn_list': [10, 13, 16],
  'material': {'type': 'material_constant_a',
   'tag': 201,
   'value': 1,
   'name': 'entire_domain',
   'a_value': 1},
  'source': {'type': 'analytical_source_fx', 'value': 0}}]

#### Material Propriety, $a$

In [10]:
a = [element['material']['a_value'] for element in mesh_data['conn_data']]
print("Material 'a' values: \n", np.array(a))

Material 'a' values: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


#### Source: Analytical source, $f(x)$

In [11]:
Fx = [element['source']['value'] for element in mesh_data['conn_data']]
print("Source vector Fx: \n", np.array(Fx))

Source vector Fx: 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


#### Linear Jacobian

In [12]:
Je = matrices_assembly.jacobian(e=0, mesh_data=mesh_data, element_type=ELEMENT, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_0:\n", Je)
print("Determinant of the Jacobian matrix for element e_0:", Jdet)

Jacobian matrix for element e_0:
 [[-0.39164173 -0.71793226]
 [ 0.11783358 -0.59584406]]
Determinant of the Jacobian matrix for element e_0: 0.31795392666059685


#### Local Elements $e_1$

In [13]:
Ae, Fe, areae = matrices_assembly.linear_local_matrices(e=0, mesh_data=mesh_data)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", Fe)
print("Area of element e_1: ", areae)

Local stiffness matrix for element e_1:
 [[ 0.43162011  0.01999067 -0.45161078]
 [ 0.01999067  0.58013893 -0.6001296 ]
 [-0.45161078 -0.6001296   1.05174038]]
Local load vector for element e_1:
 [[0.]
 [0.]
 [0.]]
Area of element e_1:  0.15897696333029843


#### Global “stiffness” matrix, $A_g$

In [14]:
Ag, fg = matrices_assembly.get_global_matrix(mesh_data, ELEMENT)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (17, 17)


### Imposition of Boundary Conditions

In [15]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")
print("Free nodes: \n", np.array(free_nodes))
print("Dirichlet nodes: \n", np.array(dirichlet_nodes))
print("Dirichlet nodes values: \n", np.array(dirichlet_values))

The entire domain has 17 nodes: 5 free nodes and 12 Dirichlet nodes.
Free nodes: 
 [13 14 15 16 17]
Dirichlet nodes: 
 [ 1  2  3  4  5  6  7  8  9 10 11 12]
Dirichlet nodes values: 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### `assign_dirichlet_potential()`

In [16]:
potential_function = lambda x, y: ps.potential_and_gradient(x, y)[0]
boundary_conditions.assign_dirichlet_potential(mesh_data, potential_function)
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
mesh_data['nodes_data']

[{'TagID': 1,
  'global_coord': (0.0, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 {'TagID': 2,
  'global_coord': (0.0, -1.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 {'TagID': 3,
  'global_coord': (1.0, -1.0),
  'bc': {'type': 'Dirichlet', 'value': 0.6299605249474365}},
 {'TagID': 4,
  'global_coord': (1.0, 1.0),
  'bc': {'type': 'Dirichlet', 'value': 1.2599210498948732}},
 {'TagID': 5,
  'global_coord': (-1.0, 1.0),
  'bc': {'type': 'Dirichlet', 'value': 0.629960524947437}},
 {'TagID': 6,
  'global_coord': (-1.0, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 1.2246467991473532e-16}},
 {'TagID': 7,
  'global_coord': (0.0, -0.2305846405365542),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 {'TagID': 8,
  'global_coord': (0.0, -0.5519932425989125),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 {'TagID': 9,
  'global_coord': (1.0, -2.752797989558076e-12),
  'bc': {'type': 'Dirichlet', 'value': 0.866025403783521}},
 {'TagID': 10,
  'global_coord': (2.752797989558076e-12, 1

### Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [17]:
global_to_reduced = {node: i for i, node in enumerate(
        idx_n for idx_n in range(1, len(nodes)+1) if idx_n not in dirichlet_nodes
    )}
print("Global to reduced nodes mapping:"); global_to_reduced

Global to reduced nodes mapping:


{13: 0, 14: 1, 15: 2, 16: 3, 17: 4}

## Processor Module
### Asymmetric Global matrix $A_g$ with boundary conditions

In [18]:
Ag, fg = boundary_conditions.apply_simple_dirichlet(Ag, fg, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (17, 17)


### Global Potential Vector

In [19]:
# Make sure Ag is in CSR format for efficiency
# Convert fg to a dense format or keep it sparse as needed
# Solve the linear system Ag * u = fg
u = spsolve(Ag.tocsr(), fg.toarray())
print("The solution vector u is: \n", u)

The solution vector u is: 
 [0.00000000e+00 0.00000000e+00 6.29960525e-01 1.25992105e+00
 6.29960525e-01 1.22464680e-16 0.00000000e+00 0.00000000e+00
 8.66025404e-01 8.66025404e-01 8.24074912e-17 4.60501556e-17
 2.44532100e-01 2.43987077e-01 4.99327889e-01 5.04675082e-01
 8.31677996e-01]


### Reduced Global matrix $A_{gr}$ with boundary conditions

In [20]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, ELEMENT)
print("Reduced global matrix shape:", Agr.shape)
print("Reduced global matrix: \n", Agr)
print("Reduced force vector: \n", fgr)

Reduced global matrix shape: (5, 5)
Reduced global matrix: 
   (0, 0)	4.333772835004987
  (0, 3)	-0.777265974537105
  (1, 1)	4.313831477824053
  (1, 2)	-0.7040590023883485
  (2, 1)	-0.7040590023883485
  (2, 2)	3.828327007949706
  (2, 3)	-0.8017755903637804
  (2, 4)	-0.8909770334083817
  (3, 0)	-0.777265974537105
  (3, 2)	-0.8017755903637804
  (3, 3)	3.8210177591383294
  (3, 4)	-0.8165085500261808
  (4, 2)	-0.8909770334083817
  (4, 3)	-0.8165085500261808
  (4, 4)	4.077741722128871
Reduced force vector: 
   (0, 0)	0.6674798031287459
  (1, 0)	0.7009628383613447
  (2, 0)	0.5941669901908456
  (3, 0)	0.6588848608226595
  (4, 0)	2.534406862910594


### Global Potential Vector

In [21]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)
print("The reduced solution vector ur is: \n", ur)
print("The global potential vector V is: \n", potential_u)

The reduced solution vector ur is: 
 [0.2445321  0.24398708 0.49932789 0.50467508 0.831678  ]
The global potential vector V is: 
 [0.00000000e+00 0.00000000e+00 6.29960525e-01 1.25992105e+00
 6.29960525e-01 1.22464680e-16 0.00000000e+00 0.00000000e+00
 8.66025404e-01 8.66025404e-01 8.24074912e-17 4.60501556e-17
 2.44532100e-01 2.43987077e-01 4.99327889e-01 5.04675082e-01
 8.31677996e-01]


### Solution at each element

In [22]:
u_list = []
for e in range(len(conn)):
    uh_values = [potential_u[id - 1] for id in conn[e]]
    u_list.append(uh_values)
print("The solution u for Node Element is: \n", np.array(u_list))

The solution u for Node Element is: 
 [[8.66025404e-01 2.44532100e-01 5.04675082e-01]
 [2.43987077e-01 8.66025404e-01 4.99327889e-01]
 [6.29960525e-01 8.66025404e-01 2.43987077e-01]
 [8.66025404e-01 6.29960525e-01 2.44532100e-01]
 [0.00000000e+00 4.99327889e-01 5.04675082e-01]
 [2.44532100e-01 0.00000000e+00 5.04675082e-01]
 [0.00000000e+00 2.43987077e-01 4.99327889e-01]
 [8.66025404e-01 1.25992105e+00 8.31677996e-01]
 [1.25992105e+00 8.66025404e-01 8.31677996e-01]
 [0.00000000e+00 0.00000000e+00 2.43987077e-01]
 [8.24074912e-17 4.60501556e-17 2.44532100e-01]
 [5.04675082e-01 4.99327889e-01 8.31677996e-01]
 [4.99327889e-01 8.66025404e-01 8.31677996e-01]
 [8.66025404e-01 5.04675082e-01 8.31677996e-01]
 [0.00000000e+00 0.00000000e+00 2.43987077e-01]
 [4.60501556e-17 0.00000000e+00 2.44532100e-01]
 [0.00000000e+00 6.29960525e-01 2.43987077e-01]
 [0.00000000e+00 0.00000000e+00 2.43987077e-01]
 [6.29960525e-01 1.22464680e-16 2.44532100e-01]
 [1.22464680e-16 8.24074912e-17 2.44532100e-01]]


In [23]:
xi_master = (1,0)
element = 13
u_at_node = matrices_assembly.isomapping_to_global_coordinates(u_list[element-1], xi_master, ELEMENT)
print(f"Element e_{element}: Potential 'u' interpolate at xi = {xi_master} --> u(xi): {u_at_node[0]}")

Element e_13: Potential 'u' interpolate at xi = (1, 0) --> u(xi): 0.866025403783521


## Post-Processor

In [24]:
graph_results.approximate_fem_solution(ELEMENT, mesh_data, potential_u, model='L')
graph_results.L_analytical_solution(potential_function)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p13_L_domain_poisson\pos_processing\pictures\L_domain_solution_Triangle1.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p13_L_domain_poisson\pos_processing\pictures\L_analytical_solution.svg


### Figure 2: Scalar Field Distribution on a Numeric Triangular Mesh
<figure>
    <img src="pos_processing/pictures/L_domain_solution_Triangle1.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

### Figure 3: Analytical Scalar Field Distribution
<figure>
    <img src="pos_processing/pictures/L_analytical_solution.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.